In [53]:
# Import the dependencies.

import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.

city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Bluff,NZ,2020-04-06 00:32:51,-46.60,168.33,60.01,90,100,5.01,light rain,0.009843,0.0
1,1,Hobart,AU,2020-04-06 00:32:51,-42.88,147.33,57.00,54,75,11.41,broken clouds,0.009843,0.0
2,2,Ucluelet,CA,2020-04-06 00:32:51,48.93,-125.55,54.00,51,20,13.87,few clouds,0.009843,0.0
3,3,Teguldet,RU,2020-04-06 00:32:51,57.31,88.17,29.50,94,2,9.55,clear sky,0.009843,0.0
4,4,Cherskiy,RU,2020-04-06 00:32:52,68.75,161.30,5.23,90,0,14.23,clear sky,0.009843,0.0


In [14]:
# Configure gmaps to use the Google API key.

gmaps.configure(api_key = g_key)

In [24]:
# Ask the customer to add a minimum and maximum temperature value.
# Ask the customer to add if they want it to be raining or snowing.

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = (input("Do you want it to be raining? (yes/no) "))
snowing = (input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [25]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
7,7,Kattivakkam,IN,2020-04-06 00:32:52,13.22,80.32,84.20,89,40,5.82,scattered clouds,0.009843,0.0
8,8,Mahebourg,MU,2020-04-06 00:32:52,-20.41,57.70,78.80,74,40,9.17,scattered clouds,0.009843,0.0
9,9,Vaini,TO,2020-04-06 00:32:52,-21.20,-175.20,80.60,65,75,6.93,broken clouds,0.009843,0.0
11,11,Ixtapa,MX,2020-04-06 00:31:42,20.70,-105.20,78.80,74,75,9.17,broken clouds,0.009843,0.0
12,12,Bambous Virieux,MU,2020-04-06 00:32:53,-20.34,57.76,78.80,74,40,9.17,scattered clouds,0.009843,0.0
14,14,Arraial Do Cabo,BR,2020-04-06 00:32:53,-22.97,-42.03,75.29,71,72,9.53,broken clouds,0.009843,0.0
15,15,Bonthe,SL,2020-04-06 00:32:53,7.53,-12.51,82.02,85,1,5.99,clear sky,0.009843,0.0
17,17,Balabac,PH,2020-04-06 00:32:53,7.99,117.06,82.76,73,50,11.77,scattered clouds,0.009843,0.0
22,22,Atuona,PF,2020-04-06 00:32:54,-9.80,-139.03,81.61,76,12,14.05,few clouds,0.009843,0.0
29,29,Port-Gentil,GA,2020-04-06 00:32:56,-0.72,8.78,81.54,86,7,9.55,clear sky,0.009843,0.0


In [26]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kattivakkam,IN,84.20,scattered clouds,13.22,80.32,
8,Mahebourg,MU,78.80,scattered clouds,-20.41,57.70,
9,Vaini,TO,80.60,broken clouds,-21.20,-175.20,
11,Ixtapa,MX,78.80,broken clouds,20.70,-105.20,
12,Bambous Virieux,MU,78.80,scattered clouds,-20.34,57.76,
14,Arraial Do Cabo,BR,75.29,broken clouds,-22.97,-42.03,
15,Bonthe,SL,82.02,clear sky,7.53,-12.51,
17,Balabac,PH,82.76,scattered clouds,7.99,117.06,
22,Atuona,PF,81.61,few clouds,-9.80,-139.03,
29,Port-Gentil,GA,81.54,clear sky,-0.72,8.78,


In [27]:
# Set parameters to search for a hotel.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [28]:
# Iterate through the DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")



Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kattivakkam,IN,84.20,scattered clouds,13.22,80.32,St. Joseph Thirumana mandapam
8,Mahebourg,MU,78.80,scattered clouds,-20.41,57.70,Shandrani Beachcomber Resort & Spa
9,Vaini,TO,80.60,broken clouds,-21.20,-175.20,Keleti Beach Resort
11,Ixtapa,MX,78.80,broken clouds,20.70,-105.20,Container Inn Puerto Vallarta
12,Bambous Virieux,MU,78.80,scattered clouds,-20.34,57.76,Casa Tia Villa
14,Arraial Do Cabo,BR,75.29,broken clouds,-22.97,-42.03,Pousada Porto Praia
15,Bonthe,SL,82.02,clear sky,7.53,-12.51,Bonthe Holiday Village
17,Balabac,PH,82.76,scattered clouds,7.99,117.06,Sing and Swing Lodge
22,Atuona,PF,81.61,few clouds,-9.80,-139.03,Villa Enata
29,Port-Gentil,GA,81.54,clear sky,-0.72,8.78,Fengshui Residence


In [49]:
# Create the output csv file.

output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the city_data into the csv file.
hotel_df.to_csv(output_data_file)

In [31]:
# Add a heatmap of temperature for the vacation spots.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [54]:
# Add a heatmap of temperature for the vacation spots and marker for each city.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [33]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [44]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [45]:
# Store the DataFrame Row.

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [46]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))